    This file is a part of the lor_neuro_rat project.
    Copyright (C) 2019 anonimous

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

    Please contact with me by E-mail: shkolnick.kun@gmail.com

# Подготовка удаленных сообщений и причин удаления для разметки.

In [23]:
import pandas as pd
#Загрузка
df = pd.read_pickle('data/all_data_as_dataframe.pkl')
#Сортировка
df.sort_values(by=['DelReason'], inplace=True, ascending=False)
df.reset_index(inplace=True, drop=True)

print(df.head())

                         TopId             MsgId                      Creator  \
0  /forum/development/14914210  comment-14914456  /people/zvezdochiot/profile   
1  /forum/development/14878132  comment-14878851  /people/zvezdochiot/profile   
2  /forum/development/14891775  comment-14891818  /people/zvezdochiot/profile   
3      /forum/general/15012556  comment-15015113  /people/zvezdochiot/profile   
4      /forum/desktop/14922772  comment-14922873  /people/zvezdochiot/profile   

                            Time                                   SrcLink  \
0  2019-04-04T20:27:19.635+03:00  /forum/development/14914210?cid=14914453   
1  2019-03-19T13:28:47.715+03:00  /forum/development/14878132?cid=14878244   
2  2019-03-25T15:58:41.011+03:00  /forum/development/14891775?cid=14891804   
3  2019-05-26T19:30:25.860+03:00      /forum/general/15012556?cid=15015099   
4  2019-04-08T23:35:13.701+03:00      /forum/desktop/14922772?cid=14922866   

                                           D

In [24]:
import re
#Удаленные сообщения
DeletedMsg = df[df['DelReason'] != '']
#Создатель
Creator = DeletedMsg['Creator'].apply(lambda x: x.split('/')[2] if x else x)
#Кто удалил
DeletedBy = DeletedMsg['DelReason'].apply(lambda x: re.sub('Сообщение удалено ', '', x).split()[0] if x else '')
#Удаленные модераторами
DeletedByMod = DeletedMsg[Creator != DeletedBy]

In [25]:
DeletedByMod.head()


,TopId,MsgId,Creator,Time,SrcLink,DelReason,Txt,Code,Quotes
159,/forum/admin/14745764,comment-14746925,,2019-01-19T13:51:36.077+03:00,,Сообщение удалено unclown по причине (0),"[актуальный список на сайте роскомсвободы, мож...",[],[]
160,/forum/talks/14747012,comment-14747032,,2019-01-19T14:35:14.317+03:00,,Сообщение удалено unclown по причине (0),"[ну надо сказать, с BBOS у них получилось]",[],[]
161,/forum/talks/14743781,comment-14743795,,2019-01-18T00:45:06.955+03:00,,Сообщение удалено unclown по причине (0),[],[echo sha|shasum\n355af72b918b1409a3b0ed2002bc...,[]
162,/forum/talks/14715993,comment-14716275,,2019-01-06T01:05:29.571+03:00,/forum/talks/14715993?cid=14716263,Сообщение удалено unclown по причине (0),"[ну да, F-150 — того же класса, что X5, без ба...",[],[]
240,/forum/development/14918675,comment-14926479,,2019-04-10T22:14:03.501+03:00,/forum/development/14918675?cid=14926444,Сообщение удалено tailgunner по причине Клоуна...,"[Возвращаясь к пизоморфным трифункторам, легко...",[],[]


In [26]:
MsgOfInterest = DeletedByMod.copy()

Reason = MsgOfInterest['DelReason'].apply(lambda x: x.split('по причине')[1])
Score = MsgOfInterest['DelReason'].apply(lambda x: int(x.split('по причине')[1].split()[-1].split('(')[1].split(')')[0]))

In [27]:
#Описания причин удаления, дальше будем размечать lookup table вручную
reasons = []
for r in Reason:
    if r not in reasons:
        reasons.append(r)
reasons.sort()

reason_lut = pd.DataFrame({'Reason':reasons, 'Class':[0]*len(reasons)})
reason_lut.to_csv('data/reasons_lut.csv', index=False, sep=';')

In [28]:
print(len(MsgOfInterest))
print(len(Reason))
print(len(Score))

6360
6360
6360


In [29]:
MsgOfInterest['Reason'] = Reason.values
MsgOfInterest['Score'] = Score.values
MsgOfInterest.to_pickle('data/MsgOfInterestDataFrame.pkl')